In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pwd

'/content'

In [3]:
%cd drive/MyDrive/2022DimitriosGagatsis_Thesis/src_video_lighting

/content/drive/MyDrive/2022DimitriosGagatsis_Thesis/src_video_lighting


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import sys
# sys.path.append('pytorchvideo')
from pytorchvideo.data import LabeledVideoDataset

In [6]:
from glob import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Data Augmentation Process

In [7]:
# Augmentation process
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from torch.utils.data import DataLoader

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,

    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [8]:
video_transform = Compose([
    ApplyTransformToKey(key='video',
        transform = Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x/255),
        NormalizeVideo(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
        RandomShortSideScale(min_size=256, max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5)
    ]),
    ),
])

# Train Data Loader

In [9]:
# # Labeled video dataset
# train_dataset = labeled_video_dataset('data_temporal/train_root',
#                                       clip_sampler=make_clip_sampler('random', 2),
#                                         transform = video_transform, decode_audio=False)


# train_loader = DataLoader(train_dataset, batch_size=1, num_workers=0, pin_memory=True)

In [10]:
# train_batch = next(iter(train_loader))

In [11]:
# train_batch.keys()
# len(train_batch)

In [12]:


# train_batch['video'].shape, train_batch['label'].shape

In [13]:
# train_batch['label']

In [14]:
# Validation Data Loader

In [15]:
# val_dataset = labeled_video_dataset('../data_temporal/val_root',
#                                       clip_sampler=make_clip_sampler('random', 2),
#                                         transform = video_transform, decode_audio=False)


# val_loader = DataLoader(train_dataset, batch_size=5, num_workers=0, pin_memory=True)

In [16]:
# val_batch = next(iter(val_loader))

In [17]:
# val_batch.keys()

In [18]:
# val_batch['video'].shape, val_batch['label'].shape

# Model Architecture

In [19]:
import torch.nn as nn
import torch
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [20]:
# video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
# Other models (https://github.com/facebookresearch/pytorchvideo/tree/main/pytorchvideo/models/hub): 
# efficient_x3d_s
# c2d_r50
# i3d_r50
# slow_r50
# slowfast_r50
# slowfast_r101
# slowfast_16x8_r101_50_50
# csn_r101
# r2plus1d_r50
# x3d_xs
# x3d_s
# x3d_m
# x3d_l

In [60]:
# model.py
class VideoModel(LightningModule):
  def __init__(self):
      super(VideoModel, self).__init__()
      # Model Architecture
      self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
      self.relu = nn.ReLU()
      self.linear = nn.Linear(400, 12)
      
      self.lr = 1e-3
      
      self.batch_size = 3
      self.numworker= 2 # If decrease it will be slower computation time
      
      # Evaluation Metric
      self.metric = torchmetrics.Accuracy(task='multiclass',   num_classes=12)
      
      # Loss Function
      # self.criterion = nn.BCEWithLogitsLoss()
      self.criterion = nn.CrossEntropyLoss()
      

  def forward(self, x):
      x = self.video_model(x)
      x = self.relu(x)
      x = self.linear(x)
      return x

  def configure_optimizers(self):
      opt = torch.optim.AdamW(params = self.parameters(), lr = self.lr)
      scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
      return {'optimizer': opt, 'lr_scheduler': scheduler}


  def train_dataloader(self):
      dataset = labeled_video_dataset('data_temporal/train_root',
                                    clip_sampler=make_clip_sampler('random', 2),
                                      transform = video_transform, decode_audio=False)


      loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
      return loader

  def training_step(self, batch, batch_idx):
      video, label = batch['video'], batch['label']
      out = self(video) # or self.forward(video)
      loss = self.criterion(out, label)
      metric = self.metric(out, label.to(torch.int64))
      return {'loss': loss, 'metric': metric.detach()}


  def on_train_epoch_end(self, outputs):
      loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
      metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
      self.log('training_loss', loss)
      self.log('trainng_metric', metric)


  def val_dataloader(self):
      dataset = labeled_video_dataset('data_temporal/val_root',
                                    clip_sampler=make_clip_sampler('random', 2),
                                      transform = video_transform, decode_audio=False)


      loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
      return loader

  def validation_step(self, batch, batch_idx):
      video, label = batch['video'], batch['label']
      out = self(video) # or self.forward(video)
      loss = self.criterion(out, label)
      metric = self.metric(out, label.to(torch.int64))
      return {'loss': loss, 'metric': metric.detach()}


  def on_validation_end(self, outputs):
      loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
      metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
      self.log('validation_loss', loss)
      self.log('validation_metric', metric)

      
  def test_dataloader(self):
      dataset = labeled_video_dataset('data_temporal/val_root',
                                    clip_sampler=make_clip_sampler('random', 2),
                                      transform = video_transform, decode_audio=False)


      loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
      return loader

  def test_step(self, batch, batch_idx):
      video, label = batch['video'], batch['label']
      out = self(video) # or self.forward(video)
      return {'label': label.detach(), 'pred': out.detach()}

  def on_test_epoch_end(self, outputs):
      label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
      pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
      pred = np.where(pred>0.5,1,0)
      print(classification_report(label, pred))
      



      
      

In [61]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath = 'checkpoints',
                                      filename='file', save_last=True)




lr_monitor = LearningRateMonitor(logging_interval='epoch')


In [62]:
model = VideoModel()
seed_everything(0)

trainer = Trainer(max_epochs = 10,
                  accelerator = 'gpu',
                      precision = '16-mixed',
                  accumulate_grad_batches = 2,
                  enable_progress_bar = True,
                  num_sanity_val_steps = 0,
                  callbacks = [lr_monitor, checkpoint_callback],
                  limit_train_batches = 2, # If you want to limit it 
                  limit_val_batches = 1, # If you want to limit it 
                 )
                  
# Next to accelerator I removed this:  , devices = -1 (this means take all GPU)


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
INFO:lightning_fabric.utilities.seed:Global seed set to 0
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


In [63]:
torch.set_float32_matmul_precision('high')
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | linear      | Linear             | 4.8 K 
3 | metric      | MulticlassAccuracy | 0     
4 | criterion   | CrossEntropyLoss   | 0     
---------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.197    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

TypeError: ignored

# Model Evaluation

In [ ]:
# Validation results
val_res = trainer.validate(model)

In [ ]:
# Testing results
trainer.test(model)

In [ ]:
from lightning_fabric.loggers import tensorboard
%load_ext tensorboard

%tensorboard --logdir logs/fit #(lightning_logs? Search for this file)

In [ ]:
# If you want to continue training from checkpoint:
# trainer.fit(model, ckpt_path='checkpoints/last.ckpt')